In [89]:
def half_adder(a, b):
  s = a ^ b
  carry = a & b
  return s, carry


def half_subtractor(a, b):
  d = a ^ b
  borrow = int(not a) & b
  return d, borrow


def full_adder(a, b, c):
  s_1, carry_1 = half_adder(a, b)
  s, carry_2 = half_adder(s_1, c)
  carry = carry_1 | carry_2
  return s, carry


def full_subtractor(a, b, c):
  d_1, borrow_1 = half_subtractor(a, b)
  d, borrow_2 = half_subtractor(d_1, c)
  borrow = borrow_1 | borrow_2
  return d, borrow


def add(A, B):
  num_bits = len(A)
  if num_bits != len(B):
    raise ValueError('strings must have same number of bits')
  S = [0] * num_bits
  a = 0
  for i in range(num_bits-1, -1, -1):
    b, c = A[i], B[i]
    s, a = full_adder(a, b, c)
    S[i] = s
  if a == 1:
    raise OverflowError()
  return S


def subtract(A, B):
  num_bits = len(A)
  if num_bits != len(B):
    raise ValueError('strings must have same number of bits')
  D = [0] * num_bits
  c = 0
  for i in range(num_bits-1, -1, -1):
    a, b = A[i], B[i]
    d, borrow = full_subtractor(a, b, c)
    if borrow == 1:
      j = i - 1
      while A[j] != 1:
        if j < 0:
          raise ValueError('negative difference')
        A[j] = 1
        j -= 1
      A[j] = 0
    D[i] = d
  return D


def multiply(A, B):
  one = [0] * (len(B) - 1) + [1]
  P = [0] * len(A)
  while any(B):
    P = add(A[:], P[:])
    B = subtract(B[:], one[:])
  return P


def divide(A, B):
  one = [0] * (len(B) - 1) + [1]
  Q = [0] * len(A)
  while True:
    try:
      A = subtract(A[:], B[:])
      Q = add(Q[:], one[:])
    except:
      break
  return Q, A

In [107]:
class and_or_latch:
  def __init__(self):
    self._SET = 0
    self._RESET = 0
    self._OUTPUT = 0
  
  def __update(self):
    self._OUTPUT = (self._OUTPUT | self.SET) & int(not self.RESET)
  
  @property
  def SET(self):
    return self._SET
  
  @property
  def RESET(self):
    return self._RESET
  
  @property
  def OUTPUT(self):
    return self._OUTPUT

  @SET.setter
  def SET(self, a):
    self._SET = a

    self.__update()

  @RESET.setter
  def RESET(self, a):
    self._RESET = a

    self.__update()


class gated_latch:
  def __init__(self):
    self._INPUT = 0
    self._WRITE_ENABLE = 0
    self._AOL = and_or_latch()
  
  def __update(self):
    self._AOL.SET = self._INPUT & self._WRITE_ENABLE
    self._AOL.RESET = int(not self._INPUT) & self._WRITE_ENABLE
  
  @property
  def INPUT(self):
    return self._INPUT
  
  @property
  def WRITE_ENABLE(self):
    return self._WRITE_ENABLE
  
  @property
  def OUTPUT(self):
    return self._AOL.OUTPUT
  
  @INPUT.setter
  def INPUT(self, a):
    self._INPUT = a

    self.__update()
  
  @WRITE_ENABLE.setter
  def WRITE_ENABLE(self, a):
    self._WRITE_ENABLE = a

    self.__update()


class register:
  def __init__(self, width=8):
    self._width = width
    self._gls = [gated_latch() for i in range(self._width)]
  
  @property
  def VALUE(self):
    return [gl.OUTPUT for gl in self._gls]
  
  @VALUE.setter
  def VALUE(self, A):
    for i in range(self._width):
      self._gls[i].WRITE_ENABLE = 1
      self._gls[i].INPUT = A[i]
      self._gls[i].WRITE_ENABLE = 0